In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from joblib import dump
from sklearn.decomposition import PCA
from sklearn.calibration import LabelEncoder
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score


df = pd.read_csv('../QualidadeAgua\waterQuality1.csv')
df.replace("#NUM!", pd.NA, inplace=True)
df = df.dropna()

Y = df['bacteria']
X = df.drop(['bacteria'], axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 42)



<>:13: SyntaxWarning: invalid escape sequence '\w'
<>:13: SyntaxWarning: invalid escape sequence '\w'
C:\Users\disrct\AppData\Local\Temp\ipykernel_10628\1538111571.py:13: SyntaxWarning: invalid escape sequence '\w'
  df = pd.read_csv('../QualidadeAgua\waterQuality1.csv')


In [ ]:
#Elastic Net
model = GridSearchCV(
    ElasticNet(fit_intercept = True),
    {
        'alpha': list(map(lambda x: x/10, range(1, 10))),
        'l1_ratio': list(map(lambda x: x/10, range(1, 10))),
    },

    n_jobs = 4
)

In [ ]:
#Decision Tree Regressor

parametros = {
    'max_depth': [ 5, 10, 20, 50, 200],# Quantidade máxima de níveis na árvore
    'min_samples_split': [5, 10]# Número mínimo de amostras necessárias para dividir um nó
}

# max_depth: Controla a profundidade máxima da árvore de decisão. 
#     Árvores muito profundas podem superajustar (overfitting), enquanto árvores muito rasas podem subajustar (underfitting).
# min_samples_split: Define o número mínimo de amostras que um nó precisa ter para poder ser dividido em dois.
#     Se for muito alto, a árvore não cresce muito; se for muito baixo, a árvore pode crescer demais e se tornar complexa.

dtr = DecisionTreeRegressor()

model = GridSearchCV(
    estimator = dtr,
    param_grid = parametros,
    cv = 5
)

In [17]:
model.fit(X_train, Y_train)
print(f'Melhores parametros: {model.best_params_}')

model = model.best_estimator_
dump(model, 'model.pkl')
print(f'Erro medio absoluto: {mean_absolute_error(Y, model.predict(X))}') #erro medio entre os valores previstos e os valores reais

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

error_test = mean_absolute_error(Y_test, Y_test_pred)
error_train = mean_absolute_error(Y_train, Y_train_pred)

print(f'Erro de treino: {error_train}')
print(f'Erro de test: {error_test}')

Melhores parametros: {'max_depth': 5, 'min_samples_split': 10}
Erro medio absoluto: 0.10997190121440953
Erro de treino: 0.10934777655844133
Erro de test: 0.11350652718270951
